# 🏠 House Price Prediction (Ames) — Linear Regression

**Author:** Ishu Singh  
**Date:** November 09, 2025

A clean, beginner-friendly workflow to predict house prices on the Ames dataset.

**Pipeline:** Setup → Cleaning → EDA → Baseline (numeric) → One‑Hot Encoding → Evaluation → Insights

> Put `train.csv` (Kaggle *House Prices - Advanced Regression Techniques*) in the same folder before running.


## 1) Setup & Data Load

In [ ]:
# Core libraries
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Style & reproducibility
sns.set(style="whitegrid")
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Load data
df = pd.read_csv('train.csv')  # Ensure train.csv is present
print("Shape:", df.shape)
df.head()

## 2) Quick Data Check

In [ ]:
print("Dtype counts:")
display(df.dtypes.value_counts())

print("\nMissing values (top 20):")
missing = df.isna().sum().sort_values(ascending=False)
display(missing.head(20))

df.describe(include='all').T.head(10)

## 3) Missing Values & Outliers
- Numeric NA → median (robust)  
- Categorical NA → mode (most frequent)  
- Outliers: remove extreme `GrLivArea` with unusually low price (stabilizes linear fit)


In [ ]:
# Numeric impute (median)
for col in df.select_dtypes(include=['int64','float64']).columns:
    if df[col].isna().any():
        df[col] = df[col].fillna(df[col].median())

# Categorical impute (mode)
for col in df.select_dtypes(include='object').columns:
    if df[col].isna().any():
        df[col] = df[col].fillna(df[col].mode()[0])

print("Remaining NaN count:", int(df.isna().sum().sum()))

# Outlier removal (domain-informed for linear models)
before = df.shape
df = df[~((df['GrLivArea'] > 4000) & (df['SalePrice'] < 300000))].copy()
after = df.shape
print("Outlier removal:", before, "->", after)

## 4) EDA Essentials

In [ ]:
# Target distribution
plt.figure(figsize=(7,4))
sns.histplot(df['SalePrice'], bins=30, kde=True)
plt.title('Distribution of SalePrice')
plt.show()

# Top numeric correlations with target
corr = df.corr(numeric_only=True)['SalePrice'].sort_values(ascending=False)
print("Top correlations with SalePrice:")
display(corr.head(12))

# Key relationships
plt.figure(figsize=(6,4))
sns.boxplot(x='OverallQual', y='SalePrice', data=df)
plt.title('OverallQual vs SalePrice')
plt.show()

plt.figure(figsize=(6,4))
sns.scatterplot(x='GrLivArea', y='SalePrice', data=df, alpha=0.6)
plt.title('GrLivArea vs SalePrice')
plt.show()

## 5) Baseline Model (Numeric-Only)

In [ ]:
# Strong numeric subset for a quick baseline
numeric_features = ['OverallQual','GrLivArea','GarageCars','TotalBsmtSF','1stFlrSF']
Xb = df[numeric_features].copy()
yb = df['SalePrice'].copy()

Xb_train, Xb_test, yb_train, yb_test = train_test_split(Xb, yb, test_size=0.2, random_state=RANDOM_SEED)

base_model = LinearRegression()
base_model.fit(Xb_train, yb_train)

yb_pred = base_model.predict(Xb_test)
base_r2 = r2_score(yb_test, yb_pred)
base_rmse = np.sqrt(mean_squared_error(yb_test, yb_pred))

print(f"Baseline — R²: {base_r2:.4f} | RMSE: {base_rmse:.2f}")

## 6) Model Improvement (One‑Hot Encoding)

In [ ]:
# One‑Hot Encoding of all categorical columns
df_ohe = pd.get_dummies(df, drop_first=True)

# Drop Id if present (identifier, not a predictor)
df_ohe = df_ohe.drop(columns=['Id'], errors='ignore')

# Features/Target from the SAME encoded frame (avoid length mismatch)
y = df_ohe['SalePrice']
X = df_ohe.drop(columns=['SalePrice'])

# Linear models can't handle NaN → safe fill
X = X.fillna(0)
y = y.fillna(0)

print("Encoded shapes:", X.shape, y.shape)

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

# Train Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)

# Predict & evaluate
y_pred = model.predict(X_test)
r2  = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Improved — R²: {r2:.4f} | RMSE: {rmse:.2f}")

## 7) Evaluation & Visuals

In [ ]:
# Actual vs Predicted
plt.figure(figsize=(6,6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6)
plt.xlabel("Actual SalePrice")
plt.ylabel("Predicted SalePrice")
plt.title("Actual vs Predicted (Improved Linear Regression)")
plt.show()

# Residuals
residuals = y_test - y_pred
plt.figure(figsize=(7,4))
sns.histplot(residuals, bins=30, kde=True)
plt.title("Residuals Distribution (Errors)")
plt.xlabel("Error = Actual - Predicted")
plt.show()

## 8) Final Insights
- Outlier removal (`GrLivArea` extreme with low price) stabilizes the linear fit.
- One‑Hot Encoding lets categorical signals (e.g., `Neighborhood`, `KitchenQual`) contribute.
- Final Linear Regression typically reaches **R² ~0.86–0.90** and **RMSE ~23k–28k** (run‑to‑run may vary).
- For this project, we intentionally keep it **Linear Regression only** to stay beginner‑friendly and interview‑oriented.
